In [1]:
import os
import yaml
import numpy as np
import pandas as pd

In [2]:
dataset_metrics = {
    'tolokers-tab': 'AP',
    'questions-tab': 'AP',
    'city-reviews': 'AP',
    'browser-games': 'accuracy',
    'hm-categories': 'accuracy',
    'web-fraud': 'AP',
    'city-roads-M': 'R2',
    'city-roads-L': 'R2',
    'avazu-devices': 'R2',
    'hm-prices': 'R2',
    'web-traffic': 'R2'
}

models = ['ResNet', 'GCN', 'GraphSAGE', 'GAT', 'GAT_sep', 'GT', 'GT_sep',
          'ResNet_plr', 'GCN_plr', 'GraphSAGE_plr', 'GAT_plr', 'GAT_sep_plr', 'GT_plr', 'GT_sep_plr']

model_names_dict = {
    'ResNet': 'ResNet',
    'GCN': 'GCN',
    'GraphSAGE': 'GraphSAGE',
    'GAT': 'GAT',
    'GAT_sep': 'GAT-sep',
    'GT': 'GT',
    'GT_sep': 'GT-sep',
    'ResNet_plr': 'ResNet-PLR',
    'GCN_plr': 'GCN-PLR',
    'GraphSAGE_plr': 'GraphSAGE-PLR',
    'GAT_plr': 'GAT-PLR',
    'GAT_sep_plr': 'GAT-sep-PLR',
    'GT_plr': 'GT-PLR',
    'GT_sep_plr': 'GT-sep-PLR'
}

# Results for classification datasets

In [3]:
datasets = ['tolokers-tab', 'questions-tab', 'city-reviews', 'browser-games', 'hm-categories', 'web-fraud']

In [4]:
df = pd.DataFrame(columns=datasets)

In [5]:
for model in models:
    results = []
    for dataset in datasets:
        if dataset == 'web-fraud' and 'plr' in model:
            results.append('OOM')
            continue
        
        val_means = []
        test_strings = []
        for num_features_preproc in ('quantile', 'standard'):
            for dropout_p in ('0', '0.2'):
                for lr in ('3e-5', '3e-4', '3e-3'):
                    file_path = f'../experiments/{dataset}/{model}_{num_features_preproc}_dropout{dropout_p}_lr{lr}_01/metrics.yaml'
                    
                    if not os.path.isfile(file_path):
                        continue
                    
                    with open(file_path) as file:
                        metrics = yaml.safe_load(file)
                    
                    if metrics['num runs'] != 5:
                        continue
                    
                    metric = dataset_metrics[dataset]
                    
                    val_mean = metrics[f'val {metric} mean']
                    val_means.append(val_mean)
                    
                    test_mean = metrics[f'test {metric} mean']
                    test_std = metrics[f'test {metric} std']
                    
                    test_string = f'{test_mean * 100:.2f} \u00B1 {test_std * 100:.2f}'
                    test_strings.append(test_string)
        
        if not val_means:
            raise RuntimeError(f'No successful experiments were found for dataset {dataset} '
                               f'and model {model_names_dict[model]}.')
        
        best_val_mean = max(val_means)
        best_val_id = val_means.index(best_val_mean)
        test_string = test_strings[best_val_id]
        results.append(test_string)
    
    df.loc[model_names_dict[model]] = results


In [6]:
df

,tolokers-tab,questions-tab,city-reviews,browser-games,hm-categories,web-fraud
ResNet,45.17 ± 0.61,84.01 ± 0.26,64.33 ± 0.32,78.82 ± 0.32,70.45 ± 0.24,14.21 ± 0.24
GCN,61.09 ± 0.38,84.92 ± 0.95,71.08 ± 0.32,79.17 ± 0.41,86.42 ± 0.31,14.65 ± 0.24
GraphSAGE,57.08 ± 0.24,85.70 ± 0.30,71.15 ± 0.27,82.56 ± 0.11,86.35 ± 0.18,20.28 ± 0.48
GAT,58.77 ± 1.00,84.44 ± 0.68,71.38 ± 0.53,82.60 ± 0.26,87.84 ± 0.23,19.95 ± 0.51
GAT-sep,61.00 ± 0.43,85.35 ± 0.32,71.13 ± 0.33,83.23 ± 0.37,89.50 ± 0.18,20.82 ± 0.28
GT,58.92 ± 0.57,83.59 ± 1.17,71.72 ± 0.23,83.29 ± 0.33,89.00 ± 0.23,20.19 ± 0.44
GT-sep,56.85 ± 1.06,84.42 ± 0.30,71.70 ± 0.39,83.04 ± 0.34,88.41 ± 0.26,20.35 ± 0.38
ResNet-PLR,48.01 ± 1.25,86.86 ± 0.22,64.04 ± 0.47,79.73 ± 0.12,70.61 ± 0.14,OOM
GCN-PLR,61.34 ± 0.47,88.44 ± 0.65,69.47 ± 0.29,80.69 ± 0.24,86.37 ± 0.24,OOM
GraphSAGE-PLR,59.04 ± 0.16,88.66 ± 0.14,71.06 ± 0.29,83.42 ± 0.33,86.47 ± 0.16,OOM


# Results for regression datasets

In [7]:
datasets = ['city-roads-M', 'city-roads-L', 'avazu-devices', 'hm-prices', 'web-traffic']

In [8]:
df = pd.DataFrame(columns=datasets)

In [9]:
for model in models:
    results = []
    for dataset in datasets:
        if dataset == 'hm-prices' and 'plr' in model:
            results.append('N/A')
            continue
        elif dataset == 'web-traffic' and 'plr' in model:
            results.append('OOM')
            continue
        
        val_means = []
        test_strings = []
        for reg_target_preproc in ('none', 'standard'):
            for num_features_preproc in ('quantile', 'standard'):
                for dropout_p in ('0', '0.2'):
                    for lr in ('3e-5', '3e-4', '3e-3'):
                        file_path = f'../experiments/{dataset}/{model}_{reg_target_preproc}_{num_features_preproc}_dropout{dropout_p}_lr{lr}_01/metrics.yaml'

                        if not os.path.isfile(file_path):
                            continue

                        with open(file_path) as file:
                            metrics = yaml.safe_load(file)

                        if metrics['num runs'] != 5:
                            continue

                        metric = dataset_metrics[dataset]

                        val_mean = metrics[f'val {metric} mean']
                        val_means.append(val_mean)

                        test_mean = metrics[f'test {metric} mean']
                        test_std = metrics[f'test {metric} std']

                        test_string = f'{test_mean * 100:.2f} \u00B1 {test_std * 100:.2f}'
                        test_strings.append(test_string)
        
        if not val_means:
            raise RuntimeError(f'No successful experiments were found for dataset {dataset} '
                               f'and model {model_names_dict[model]}.')
        
        best_val_mean = max(val_means)
        best_val_id = val_means.index(best_val_mean)
        test_string = test_strings[best_val_id]
        results.append(test_string)
    
    df.loc[model_names_dict[model]] = results


In [10]:
df

,city-roads-M,city-roads-L,avazu-devices,hm-prices,web-traffic
ResNet,70.58 ± 0.35,67.49 ± 0.09,21.60 ± 0.08,67.31 ± 0.21,73.19 ± 0.04
GCN,72.87 ± 0.21,70.92 ± 0.23,27.31 ± 0.17,77.05 ± 0.25,81.95 ± 0.08
GraphSAGE,73.35 ± 0.58,71.03 ± 0.90,27.99 ± 0.32,76.01 ± 0.47,84.04 ± 0.19
GAT,73.64 ± 0.30,71.74 ± 0.23,28.28 ± 0.54,78.02 ± 0.32,84.85 ± 0.17
GAT-sep,72.65 ± 0.24,71.17 ± 0.20,26.72 ± 0.33,77.59 ± 0.23,85.02 ± 0.14
GT,72.95 ± 0.47,69.98 ± 0.57,30.27 ± 0.26,78.44 ± 0.58,85.17 ± 0.17
GT-sep,72.14 ± 0.15,69.75 ± 1.69,26.70 ± 0.47,77.35 ± 0.30,85.12 ± 0.11
ResNet-PLR,70.03 ± 0.30,67.48 ± 0.36,19.70 ± 0.73,N/A,OOM
GCN-PLR,72.92 ± 0.36,70.91 ± 0.17,26.92 ± 0.21,N/A,OOM
GraphSAGE-PLR,72.98 ± 0.64,70.52 ± 0.79,26.61 ± 0.29,N/A,OOM
